In [37]:
import json
import os
import numpy as np
import glob
import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm

In [133]:
def toTime(time):
    year, month, day = int(time[:4]), int(time[5:7]), int(time[8:10])
    hour, minute, second = int(time[11:13]), int(time[14:16]), int(time[17:19])

    if len(time) > 19:
        microsec = int(time[20:-1])
        return datetime.datetime(year, month, day, hour, minute, second, microsec)
    else:
        return datetime.datetime(year, month, day, hour, minute, second)

In [102]:
def fileTime(time):
    year, month, day = time[:4], time[5:7], time[8:10]
    hour, minute, second = time[11:13], time[14:16], time[17:19]
    
    return year+month+day+hour+minute+second

In [152]:
def genDuration(start, ptime, stime, sample_rate, length):
    
    # time: str -> datetime.datetime
    start, ptime, stime = toTime(start), toTime(ptime), toTime(stime)
    
    # ptime, stime 減去儀器開始時間，以得到具體在哪個 sample point
    diff_p, diff_s = ptime-start, stime-start
    sec_diff_p, microsec_diff_p = diff_p.seconds, diff_p.microseconds
    sec_diff_s, microsec_diff_s = diff_s.seconds, diff_s.microseconds
    
    p_trigger_sample = int((sec_diff_p+microsec_diff_p/1e+5)*sample_rate)
    s_trigger_sample = int((sec_diff_s+microsec_diff_s/1e+5)*sample_rate)

    # 生成 ground-truth
    gt = np.zeros(length)
    gt[p_trigger_sample:s_trigger_sample] = 1
    
    return gt

In [88]:
# 速度微分 -> 加速度
# 100.0: sampling_rate
def v_to_a(z, n, e):
    d_z = np.gradient(z, 1.0/100.0)
    d_n = np.gradient(n, 1.0/100.0)
    d_e = np.gradient(e, 1.0/100.0)

    return d_z, d_n, d_e

In [173]:
def toCSV(z, n, e, gt, ori_time, station, instrument, stage):
    to_write = np.vstack((e, n, z, gt))
    ori_time = fileTime(ori_time)
   
    # filename: 地震發生時間 + 測站名 + 儀器種類
    file_name = ori_time + '_' + station + '_' + instrument
    base_path = os.path.join(stage, file_name)

    np.savetxt(base_path+'.csv', to_write, delimiter=',')

### 取地震波形:
    
    1. 先檢查 s_arrival_time 是否有效(有些 pfile 中的值=0) && 篩選震央距離或震度來取 waveforms
        
    2. 取 Z, N, E, factor, p_arrival_time, s_arrival_time
    
    3. 確定資料類型是 Velocity or Acceleration，if Velocity: 微分成 acceleration
    
    4. Label 出 p-wave duration (ground-truth)
    
    5. 匯入 csv

In [174]:
def getSeisWave(p):
    for k in p.keys():
        try:
            if 'location' in p[k].keys():
                if len(p[k]['location']) != 1:
                    n_data = len(p[k]['location'])
                    for i in range(n_data):
                        
                        # Check Stime is available, 篩選距離震央距離  TODO
                        if p[k]['DataAvailable']['Stime'][i]:
                            z = p[k]['Z'][i]
                            n = p[k]['N'][i]
                            e = p[k]['E'][i]
                            
                            z, n, e = np.array(z), np.array(n), np.array(e)
                            z, n, e = z*p[k]['factor'][i][0], n*p[k]['factor'][i][1], e*p[k]['factor'][i][2]
                            ptime, stime = p[k]['p_arrival_time'][i], p[k]['s_arrival_time'][i]
                            starttime = p[k]['starttime'][i]
                            sample_rate = p[k]['sampling_rate'][i]

                            # Label p-wave duration
                            gt = genDuration(starttime, ptime, stime, sample_rate, z.shape[0])

                            # Check data is velocity
                            if p[k]['datatype'][i] == 'Velocity':
                                z, n, e = v_to_a(z, n, e)
                              
                            # 匯入 CSV file
                            ori_time = p['ori_time']
                            instrument = p[k]['instrument'][i]

                            toCSV(z, n, e, gt, ori_time, k, instrument, 'seis')
                        else:
                            continue
                else:
                    if p[k]['DataAvailable']['Stime']:
                        z = p[k]['Z']
                        n = p[k]['N']
                        e = p[k]['E']
                        
                        z, n, e = np.array(z), np.array(n), np.array(e)
                        z, n, e = z*p[k]['factor'][0][0], n*p[k]['factor'][0][1], e*p[k]['factor'][0][2]
                        ptime, stime = p[k]['p_arrival_time'][0], p[k]['s_arrival_time'][0]
                        starttime = p[k]['starttime'][0]
                        sample_rate = p[k]['sampling_rate'][0]
                        
                        # Label p-wave duration
                        gt = genDuration(starttime, ptime, stime, sample_rate, z.shape[0])

                        # Check data is velocity
                        if p[k]['datatype'] == 'Velocity':
                            z, n, e = v_to_a(z, n, e)
                              
                        # 匯入 CSV file
                        ori_time = p['ori_time']
                        instrument = p[k]['instrument'][0]
                        
                        toCSV(z, n, e, gt, ori_time, k, instrument, 'noise')
                    else:
                        continue
        except Exception as e:
            print(e)

In [172]:
getSeisWave(p)

'str' object has no attribute 'keys'
'float' object has no attribute 'keys'
'float' object has no attribute 'keys'
'float' object has no attribute 'keys'
'float' object has no attribute 'keys'
'float' object has no attribute 'keys'
'int' object has no attribute 'keys'
'float' object has no attribute 'keys'
'float' object has no attribute 'keys'
'float' object has no attribute 'keys'
'str' object has no attribute 'keys'
'int' object has no attribute 'keys'
'str' object has no attribute 'keys'
noise/20201210181508_EHP_FBA
noise/20201210181508_LATB_BB
noise/20201210181508_FUSB_BB
noise/20201210181508_NDT_FBA
noise/20201210181508_ETL_FBA
noise/20201210181508_NACB_BB
noise/20201210181508_NNSB_BB
noise/20201210181508_YHNB_BB
noise/20201210181508_WFSB_BB
noise/20201210181508_TATO_BB
noise/20201210181508_NHY_FBA
noise/20201210181508_NOU_FBA
noise/20201210181508_BAC_FBA
noise/20201210181508_EYL_FBA
noise/20201210181508_ETM_FBA
noise/20201210181508_YM01_BB
noise/20201210181508_LXIB_BB
noise/2020

### 取 Noise 波形:
    
    1. TODO

In [ ]:
def getNoiseWave(p):
    

In [2]:
root = "/mnt/nas6/CWBDatasets/2020/"
files = os.listdir(root)

In [3]:
for f in files:
    f = open(os.path.join(root, f), 'r')
    p = json.load(f)
    #print(p['mag'])
    break